# Inference

In [1]:
import rasterio, os
import torch
import numpy as np
from tqdm import tqdm

with rasterio.open('train/images/5.tif') as multi_band_src:
    image = multi_band_src.read()
    meta = multi_band_src.meta

In [2]:
predict = np.zeros((meta['height'], meta['width']), dtype=np.float32)

model = torch.load("best_model_.pt").to('cpu')
model.eval();

In [3]:

with torch.no_grad():
    for y in tqdm(range(0, meta['height'], 256)):
        for x in range(0, meta['width'], 256):
            tile = image[:, y:y+256, x:x+256]
            if x+256 > meta['width']:
                padx = -meta['width']%256
                tile = np.pad(tile, ((0,0), (0,0), (0, padx)))
            if y+256 > meta['height']:
                pady = -meta['height']%256
                tile = np.pad(tile, ((0,0), (0, pady), (0, 0)))
            tile = torch.from_numpy(tile).float()
            pred = model(tile.unsqueeze(0))
            pred = pred[0,0].numpy()
            if x+256 > meta['width']:
                pred = pred[:, :-padx]
            if y+256 > meta['height']:
                pred = pred[:-pady, :]
            predict[y:y+256, x:x+256] = pred


100%|██████████| 10/10 [00:49<00:00,  4.93s/it]


In [4]:
os.makedirs('out', exist_ok=True)

meta['count'] = 1

with rasterio.open('out/5.tif', 'w', **meta) as fout:
    fout.write(predict, 1)